In [80]:
import cv2
import os
import random

In [81]:
# global variables
rad = 5
radiusImcrement = 3
pos = [100,100]
inc = 0

In [82]:
def drawBall(img,centerTuple,radi)->None:
    cv2.circle(img,centerTuple,radi,(0,0,255),cv2.FILLED)

def generateRandomCordinates():
    return (random.randint(0,500),random.randint(0,450))

def reset():
    pos[0],pos[1] = generateRandomCordinates()
    pass



In [83]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
while True:
    success, img = cap.read()
    rad = rad + radiusImcrement
    drawBall(img,(pos[0],pos[1]),int(rad))
    if rad>35:
        radiusImcrement = -1*(radiusImcrement)
        # reset()
    if rad<5:
        radiusImcrement = -1*(radiusImcrement)
        reset()
    cv2.imshow("Video", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()